In [1]:
import torch
print(torch.__version__)
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')
roberta.eval()  # disable dropout (or leave in train mode to finetune)

1.8.2


Using cache found in /home/jess/.cache/torch/hub/pytorch_fairseq_master


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


2022-05-12 15:16:42 | INFO | fairseq.file_utils | http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz not found in cache, downloading to /tmp/tmp75wgeg1m
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 655283069/655283069 [03:53<00:00, 2810585.17B/s]
2022-05-12 15:20:36 | INFO | fairseq.file_utils | copying /tmp/tmp75wgeg1m to cache at /home/jess/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
2022-05-12 15:20:37 | INFO | fairseq.file_utils | creating metadata file for /home/jess/.cache/torch/pytorch_fairseq/83e3a689e28e5e4696ecb0bbb05a77355444a5c8a3437e0f736d8a564e80035e.c687083d14776c1979f3f71654febb42f2bb3d9a94ff7ebdfe1ac6748dba89d2
2022-05-12 15:20:37 | INFO | fairseq.file_utils | removing temp file /tmp/tmp75wgeg1m
2022-05-12 15:20:37 | INFO | fairseq.file_utils | loading archive

2022-05-12 15:20:58 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json not found in cache, downloading to /tmp/tmpvo_zrqzu
1042301B [00:00, 1048500.05B/s]
2022-05-12 15:20:59 | INFO | fairseq.file_utils | copying /tmp/tmpvo_zrqzu to cache at /home/jess/.cache/torch/pytorch_fairseq/e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2022-05-12 15:20:59 | INFO | fairseq.file_utils | creating metadata file for /home/jess/.cache/torch/pytorch_fairseq/e2aab4d600e7568c2d88fc7732130ccc815ea84ec63906cb0913c7a3a4906a2e.0f323dfaed92d080380e63f0291d0f31adfa8c61a62cbcb3cb8114f061be27f7
2022-05-12 15:20:59 | INFO | fairseq.file_utils | removing temp file /tmp/tmpvo_zrqzu
2022-05-12 15:21:00 | INFO | fairseq.file_utils | https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe not found in cache, downloading to /tmp/tmp3skkrnht
456318B [00:00, 582061.16B/s]
2022-05-12 15:21:02 | INF

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dr

In [10]:
roberta = roberta.cuda()
tokens = roberta.encode('Hello   xxxx deisj sujns ijje suu siisjsnsn world!')
# assert tokens.tolist() == [0, 31414, 232, 328, 2]
# assert roberta.decode(tokens) == 'Hello world!'

In [11]:
last_layer_features = roberta.extract_features(tokens)

# Extract all layer's features (layer 0 is the embedding layer)
all_layers = roberta.extract_features(tokens, return_all_hiddens=False)
print(all_layers.shape)

torch.Size([1, 25, 1024])


In [14]:
import numpy as np
import pandas as pd
import h5py

df_tags = pd.read_csv("data_2022/train/train_tags.csv")
df_tags2 = pd.read_json("data_2022/test/test_tags.json")
df_tags = df_tags.merge(df_tags2, how='outer')
df_tags= df_tags.fillna('nan')

len1 = len(df_tags)
feats = []
feats2=[]
hf = h5py.File('roberta_title.h5', 'w')
hf2 = h5py.File('roberta_alltags.h5', 'w')

print('Found %d data to be processed'%len1)
for i in range(len1):
    tokens = roberta.encode(df_tags['Title'].values[i])
    feat = roberta.extract_features(tokens, return_all_hiddens=False).view(-1)
    hf.create_dataset(str(df_tags['Pid'].values[i]), data= feat.detach().cpu().numpy(), compression="gzip", compression_opts=7)
    
    tags = df_tags['Alltags'].values[i]
    if len(tags)>512:
        tags = tags[:512]
    tokens = roberta.encode(tags)
    feat = roberta.extract_features(tokens, return_all_hiddens=False).view(-1)
    hf2.create_dataset(str(df_tags['Pid'].values[i]), data= feat.detach().cpu().numpy(), compression="gzip",compression_opts=7)
    
    if i % (len1//100)==0:
        print('Progress on extracting roberta features: %.2f' % (i/len1))
    
hf.close()
hf2.close()


Found 486194 data to be processed
Progress on extracting roberta features: 0.00
Progress on extracting roberta features: 0.01
Progress on extracting roberta features: 0.02
Progress on extracting roberta features: 0.03
Progress on extracting roberta features: 0.04
Progress on extracting roberta features: 0.05
Progress on extracting roberta features: 0.06
Progress on extracting roberta features: 0.07
Progress on extracting roberta features: 0.08
Progress on extracting roberta features: 0.09
Progress on extracting roberta features: 0.10
Progress on extracting roberta features: 0.11
Progress on extracting roberta features: 0.12
Progress on extracting roberta features: 0.13
Progress on extracting roberta features: 0.14
Progress on extracting roberta features: 0.15
Progress on extracting roberta features: 0.16
Progress on extracting roberta features: 0.17
Progress on extracting roberta features: 0.18
Progress on extracting roberta features: 0.19
Progress on extracting roberta features: 0.20


In [13]:
    
hf.close()
hf2.close()